In [5]:
import pandas as pd
from pandas import Series, DataFrame
import pickle
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
#from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

In [6]:
"""=====================================================================================================================
1 读取数据
"""
path = "E:/data2/"
test_transaction = pd.read_csv(path + "test_transaction_V1.csv")
tag=pd.read_csv(path + "tag_train_new.csv")
test_operation = pd.read_csv(path + "test_operation_V1.csv")

In [7]:
"""=====================================================================================================================
2 数据处理
"""

"""【transaction数据】"""
test_transaction_UID = test_transaction.UID
## 删去test_transaction 的["UID"]
test_transaction.drop(["UID"], axis=1, inplace=True)
x_test_transaction = test_transaction.values

"""【operation数据】"""
test_operation_UID = test_operation.UID
## 删去test_operation 的["UID"]
test_operation.drop(["UID"], axis=1, inplace=True)
x_test_operation = test_operation.values

"""=====================================================================================================================
3 保存特征
"""

'=====================================================================================================================\n3 保存特征\n'

In [8]:
"""=====================================================================================================================
2 模型预测
"""
model_path="E:/data2/model/"
clf1=joblib.load(model_path + "transaction_lr_30.m")
clf2=joblib.load(model_path + "operation_lr_30.m")
"""【分类预测】"""
y_test_transaction = clf1.predict(x_test_transaction)
y_test_operation = clf2.predict(x_test_operation)
"""【概率预测】"""
y_test_transaction_proba = clf1.predict_proba(x_test_transaction)
y_test_operation_proba = clf2.predict_proba(x_test_operation)

"""=====================================================================================================================
3 概率文件处理
"""
"""【格式转化】"""
y_test_transaction_proba = DataFrame(y_test_transaction_proba)

y_test_operation_proba = DataFrame(y_test_operation_proba)

test_transaction_UID = DataFrame(test_transaction_UID)

test_operation_UID = DataFrame(test_operation_UID)

"""【概率拼接】"""
y_test_proba = pd.concat(objs=[y_test_transaction_proba, y_test_operation_proba], axis=0, sort=True)

"""【UID拼接】"""
test_UID = pd.concat(objs=[test_transaction_UID, test_operation_UID], axis=0, sort=True)

"""【添加UID】"""
y_test_proba.insert(0, "UID", test_UID)

"""【删去第1列】"""

y_test_proba.drop(y_test_proba.columns[1], axis=1, inplace=True)

"""【重命名】"""

y_test_proba.rename(columns={1: "Tag"}, inplace=True)

"""【分组并求均值】"""
means = y_test_proba.groupby('UID').mean()['Tag']
means2 = DataFrame(means.values)

"""【评估】"""

'【评估】'

In [9]:
"""【保存文件】"""
means=DataFrame(means)
means.reset_index(inplace=True)
result=means.loc[:,['UID','Tag']]
path2="E:/data2/result/"
result.to_csv(path2 + 'result_lr_30.csv', index=False)

In [11]:
"""【在训练集上评估分数】"""
"""=====================================================================================================================
1 读取数据
"""
train_transaction = pd.read_csv(path + "train_transaction_V1.csv")
train_operation = pd.read_csv(path + "train_operation_V1.csv")
"""=====================================================================================================================
2 数据处理
"""

"""【transaction数据】"""
y_train_transaction = train_transaction.Tag
train_transaction_UID=train_transaction.UID

## 删去train_transaction 的["UID","Tag"]

train_transaction.drop(["UID", "Tag"], axis=1, inplace=True)
x_train_transaction = train_transaction.values



"""【operation数据】"""
y_train_operation = train_operation.Tag
train_operation_UID=train_operation.UID
## 删去train_operation 的["UID","Tag"]
train_operation.drop(["UID", "Tag"], axis=1, inplace=True)
x_train_operation = train_operation.values


y_train_transaction_proba = clf1.predict_proba(x_train_transaction)
y_train_operation_proba = clf2.predict_proba(x_train_operation)

"""=====================================================================================================================
3 概率文件处理
"""
"""【格式转化】"""
y_train_transaction_proba = DataFrame(y_train_transaction_proba)

y_train_operation_proba = DataFrame(y_train_operation_proba)

train_transaction_UID = DataFrame(train_transaction_UID)

train_operation_UID = DataFrame(train_operation_UID)

"""【概率拼接】"""
y_train_proba = pd.concat(objs=[y_train_transaction_proba, y_train_operation_proba], axis=0, sort=True)

"""【UID拼接】"""
train_UID = pd.concat(objs=[train_transaction_UID, train_operation_UID], axis=0, sort=True)

"""【添加UID】"""
y_train_proba.insert(0, "UID", train_UID)

"""【删去第1列】"""

y_train_proba.drop(y_train_proba.columns[1], axis=1, inplace=True)

"""【重命名】"""

y_train_proba.rename(columns={1: "Tag"}, inplace=True)

"""【分组并求均值】"""
train_means = y_train_proba.groupby('UID').mean()['Tag']
train_means = DataFrame(train_means.values)
d = pd.DataFrame()
d['y'] = tag['Tag']
d['prob'] = train_means[0]
d = d.sort_values(['prob'], ascending=[0])
y = d.y
PosAll = y.value_counts()[1]
NegAll = y.value_counts()[0]
pCumsum = d['y'].cumsum()
nCumsum = np.arange(len(y)) - pCumsum + 1
pCumsumPer = pCumsum / PosAll
nCumsumPer = nCumsum / NegAll
TR1 = pCumsumPer[abs(nCumsumPer - 0.001).idxmin()]
TR2 = pCumsumPer[abs(nCumsumPer - 0.005).idxmin()]
TR3 = pCumsumPer[abs(nCumsumPer - 0.01).idxmin()]
a=0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3
print(a)

0.0
